In [4]:
import pandas as pd
import sqlalchemy
import os
from sklearn import tree
from sklearn import metrics

In [5]:
# Pega o passo absoluto de um determinado caminho que será passado, no caso, file. Que é o caminho absoluto desse script pyton.
EP_DIR = os.path.abspath('.')
SRC_DIR = os.path.dirname( EP_DIR )
BASE_DIR = os.path.dirname( SRC_DIR )
DATA_DIR = os.path.join( SRC_DIR, 'data')
DATA_DIR

'c:\\Users\\kaue.mendes.FBIZ\\Desktop\\Estudos Data Science\\d-tona\\data'

In [6]:
### Importar query
# É criada a função import_query, para não precisar digitar todas as linhas abaixo para ler um arquivo, então é criada uma função para fazer essa leitura e retorna o resultado, que no caso é uma query

def import_query( path, **kwards ) :
    with open( path, 'r', **kwards ) as file_open:
        result = file_open.read()
    return result

query = import_query(os.path.join(EP_DIR,'create_safra.sql' ) )
query

"/*Visualizando os clientes que realizaram venda nos próximos 3 meses após a base estudada*/\n\nSELECT\n    t1.*\n,   case when t2.seller_id is not null then 1 else 0 end as flag_model\nFROM\n    tb_book_sellers as t1\nLEFT JOIN\n    (\n    SELECT DISTINCT t2.seller_id\n    FROM tb_orders t1\n    LEFT JOIN tb_order_items t2 ON t1.order_id = t2.order_id\n    WHERE t1.order_approved_at BETWEEN '2017-04-01' AND '2017-07-01'\n    AND order_status = 'delivered'\n    ) as t2\nON t1.seller_id = t2.seller_id\nORDER BY flag_model\n"

In [7]:
# Conectando com o Banco de Dados

def connect_db():
    return sqlalchemy.create_engine("sqlite:///" + os.path.join( DATA_DIR, 'olist.db') )

con = connect_db()

In [15]:
# Imprimindo o book de variáveis em um DF
df = pd.read_sql(query, con)
df

,seller_city,seller_state,seller_id,avg_review_score,idade_base_dias,idade_base_mes,qtde_dias_utl_venda,qtde_mes_ativacao,prop_ativacao,prop_atraso,...,qtde_fashion_calcados,qtde_eletrodomesticos_2,qtde_construcao_ferramentas_jardim,qtde_agro_industria_e_comercio,qtde_artes,qtde_pcs,qtde_sinalizacao_e_seguranca,qtde_construcao_ferramentas_seguranca,qtde_artigos_de_natal,flag_model
0,curitiba,PR,48efc9d94a9834137efd9ea76b065a38,5.000000,98.019711,4,64,2,0.500000,0,...,0,0,0,0,0,0,0,0,0,0
1,tiete,SP,817f85dbb65aa3e70831d90fe75cdf89,4.545455,178.563600,6,16,4,0.666667,0,...,0,0,0,0,0,0,0,0,0,0
2,brasilia,DF,c9a06ece156bb057372c68718ec8909b,3.363636,175.474919,6,75,2,0.333333,0,...,0,0,0,0,0,0,0,0,0,0
3,curitiba,PR,ecccfa2bb93b34a3bf033cc5d1dcdc69,2.846154,197.488449,7,171,1,0.166667,0,...,0,0,0,0,0,0,0,0,0,0
4,limeira,SP,b499c00f28f4b7069ff6550af8c1348a,3.333333,178.569873,6,24,4,0.666667,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647,votorantim,SP,fd312b6bf05efac6c3772d5b52205d8a,5.000000,2.253241,1,2,1,1.000000,0,...,0,0,0,0,0,0,0,0,0,1
648,campinas,SP,fe2032dab1a61af8794248c8196565c9,4.000000,23.477211,1,23,1,1.000000,0,...,0,0,0,0,0,0,0,0,0,1
649,caratinga,MG,fe4cd9461203cee790d36792420b310f,5.000000,18.659028,1,18,1,1.000000,0,...,0,0,0,0,0,0,0,0,0,1
650,sao paulo,SP,ff314fa6033cc68ec451c47aee2d6ba4,5.000000,65.609421,3,65,1,0.333333,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
df.head(11)


,seller_city,seller_state,seller_id,avg_review_score,idade_base_dias,idade_base_mes,qtde_dias_utl_venda,qtde_mes_ativacao,prop_ativacao,prop_atraso,...,qtde_fashion_calcados,qtde_eletrodomesticos_2,qtde_construcao_ferramentas_jardim,qtde_agro_industria_e_comercio,qtde_artes,qtde_pcs,qtde_sinalizacao_e_seguranca,qtde_construcao_ferramentas_seguranca,qtde_artigos_de_natal,flag_model
0,curitiba,PR,48efc9d94a9834137efd9ea76b065a38,5.000000,98.019711,4,64,2,0.500000,0,...,0,0,0,0,0,0,0,0,0,0
1,tiete,SP,817f85dbb65aa3e70831d90fe75cdf89,4.545455,178.563600,6,16,4,0.666667,0,...,0,0,0,0,0,0,0,0,0,0
2,brasilia,DF,c9a06ece156bb057372c68718ec8909b,3.363636,175.474919,6,75,2,0.333333,0,...,0,0,0,0,0,0,0,0,0,0
3,curitiba,PR,ecccfa2bb93b34a3bf033cc5d1dcdc69,2.846154,197.488449,7,171,1,0.166667,0,...,0,0,0,0,0,0,0,0,0,0
4,limeira,SP,b499c00f28f4b7069ff6550af8c1348a,3.333333,178.569873,6,24,4,0.666667,0,...,1,0,0,0,0,0,0,0,0,0
5,sao paulo,SP,4e8dacf3d38f281ae26c3e0321d92d88,2.888889,63.353519,3,17,3,1.000000,0,...,0,0,0,0,0,0,0,0,0,0
6,campinas,SP,ffff564a4f9085cd26170f4732393726,3.250000,175.248843,6,65,2,0.333333,0,...,0,0,0,0,0,0,0,0,0,0
7,entre rios do oeste,PR,4d600e08ecbe08258c79e536c5a42fee,4.428571,176.332465,6,16,4,0.666667,0,...,0,0,0,0,0,0,0,0,0,0
8,marilia,SP,5151aea44289d6c6b090ee31c2132508,1.714286,42.345150,2,38,1,0.500000,0,...,0,0,0,0,0,0,0,0,0,0
9,belo horizonte,MG,54aa8463e87776b5612f4bdc4bd5d4de,3.857143,171.789572,6,37,2,0.333333,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df.columns

Index(['seller_city', 'seller_state', 'seller_id', 'avg_review_score',
       'idade_base_dias', 'idade_base_mes', 'qtde_dias_utl_venda',
       'qtde_mes_ativacao', 'prop_ativacao', 'prop_atraso',
       'avg_tempo_entrega_est', 'receita_total', 'avg_vl_venda',
       'avg_vl_venda_mes', 'avg_vl_venda_mes_ativado', 'qtde_vendas',
       'qtde_produto', 'qtde_produto_dst', 'avg_vl_produto',
       'avg_qtde_produto_venda', 'qtde_cama_mesa_banho', 'qtde_beleza_saude',
       'qtde_esporte_lazer', 'qtde_moveis_decoracao',
       'qtde_informatica_acessorios', 'qtde_utilidades_domesticas',
       'qtde_relogios_presentes', 'qtde_telefonia', 'qtde_ferramentas_jardim',
       'qtde_automotivo', 'qtde_brinquedos', 'qtde_cool_stuff',
       'qtde_perfumaria', 'qtde_bebes', 'qtde_eletronicos', 'qtde_papelaria',
       'qtde_fashion_bolsas_e_acessorios', 'qtde_pet_shop',
       'qtde_moveis_escritorio', 'qtde_consoles_games',
       'qtde_malas_acessorios', 'qtde_construcao_ferramentas_construc

In [27]:
#Escolhendo as variáveis para ser usada:

columns = df.columns.tolist()

# Variáveis para serem removidas
to_remove = ['seller_id', 'seller_city']

# Variável alvo, target, resposta
target = 'flag_model'

# Remove de fato as variáveis
for i in to_remove + [target]:
    columns.remove(i)

# Defini tipos de variáveis
cat_features = df[ columns ].dtypes[ df[ columns ].dtypes == 'object'].index.tolist()
num_features = list( set( columns ) - set( cat_features) )

# Treinando o algoritmo de arvore de decisao
clf = tree.DecisionTreeClassifier(max_depth=10)
clf.fit( df[num_features], df[target] )

y_pred = clf.predict( df[num_features] )
y_prob = clf.predict_proba( df[num_features] )

# Exibe matriz de confusão
metrics.confusion_matrix( df[target], y_pred )

# Variáveis significativas
features_importance =  pd.Series( clf.feature_importances_, index = num_features)
features_importance.sort_values(ascending = False)[:20]

# qtde_dias_ult_venda essa variável é injusta, pois se um cliente já está ao longo tempo sem vender a chance dele não vender nos próximos 3 dias é grande

qtde_dias_utl_venda                 0.449158
avg_review_score                    0.080445
avg_vl_produto                      0.074949
idade_base_dias                     0.074021
avg_vl_venda_mes_ativado            0.060204
avg_tempo_entrega_est               0.044382
avg_vl_venda_mes                    0.039976
receita_total                       0.027756
qtde_produto                        0.020838
avg_vl_venda                        0.020522
qtde_utilidades_domesticas          0.015637
qtde_eletrodomesticos               0.015465
qtde_fashion_bolsas_e_acessorios    0.013436
qtde_artes                          0.008206
qtde_papelaria                      0.007875
qtde_bebes                          0.007544
qtde_moveis_decoracao               0.007293
avg_qtde_produto_venda              0.006879
qtde_esporte_lazer                  0.006707
qtde_mes_ativacao                   0.005902
dtype: float64